<a href="https://colab.research.google.com/github/Lilly-yang/UCA--Machine_Learning_and_Computer_Vision/blob/main/Project_Automatic_Signal_Detector_1_2_tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatic Signal Detector**
Task 1 - Task 2

## README




The goal of this progress is to automatically detect face.
- Reads the camera in Google Colab and run facedetect
- Reduce the computation time of Facedetect


### Getting Started
There is a model (.xml file) in my Google driver will be used in this progress. Make sure you are mounted [my Google driver](https://drive.google.com/drive/folders/1SyAjYyn7sxlJULhwh1kkbA71xfAGkk2A?usp=sharing).

_haarcascade_frontalface_alt.xml_


### Running the progress
1. Running cells in each part
2. Runing cell of `main()` to start face detection


### Authors
Li YANG   li.yang-li@etu.univ-cotedazur.fr

## Mounted Google Driver

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Camera Capture
Using a webcam by JavaScript code to capture images in Google Colab for processing on the runtime.

PS: this cell of code (except comments) were copied from our course webpage.

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode,b64encode
from PIL import Image
import io

# capture camera image by JavaScript code in Google Colab
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x

## Face Detection
This progress use haar cascades.

USAGE: `facedetect.py [--cascade <cascade_fn>] [--nested-cascade <cascade_fn>] [<video_source>]`

Below are some import and functions copied from GitHub

In [ ]:
## Python 2/3 compatibility
from __future__ import print_function

import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
import sys, getopt


## Some useful functions were copied form local files
## Function for timing
def clock():
    return cv.getTickCount() / cv.getTickFrequency()

## Function for imformation presentation of captured image
def draw_str(dst, target, s):
    x, y = target
    cv.putText(dst, s, (x+1, y+1), cv.FONT_HERSHEY_PLAIN, 1.0, (0, 0, 0), thickness = 2, lineType=cv.LINE_AA)
    cv.putText(dst, s, (x, y), cv.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), lineType=cv.LINE_AA)


## Detect face by trained model in each grid of the giving image:
##   if there is a face in the image then return the indexation of the rectangel are
##   if there is not any face in the image then return a empty list: []
def detect(img, cascade):
    rects = cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30),
                                     flags=cv.CASCADE_SCALE_IMAGE)
    if len(rects) == 0:
        return []
    rects[:,2:] += rects[:,:2]
    return rects

## Drawing a rectangle on image by giving indexation
def draw_rects(img, rects, color):
    for x1, y1, x2, y2 in rects:
        cv.rectangle(img, (x1, y1), (x2, y2), color, 2)

Main function of Face detectation:
* Face detection
* Part of Reducing time of detection

In [ ]:
def main():
    ## Loding models from Google Driver 
    cascade_fn = "/content/drive/My Drive/MLCV/haarcascade_frontalface_alt.xml"
    cascade = cv.CascadeClassifier(cv.samples.findFile(cascade_fn))

    ## initial camera caoture function
    VideoCapture()
    eval_js('create()')

    ## Main function of On-time Face Detection
    rects = []  # initial rects as [] for detect face from whole image

    while True:
      ## capture image and tralate to array d-type
      byte = eval_js('capture()')
      img = byte2image(byte)
      # print(np.shape(img))  # (480, 640, 3)

      gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
      
      # Reduce the computation time of not-first Facedetect
      if len(rects) == 0:
        # print('Detect the whole image!')
        gray = cv.equalizeHist(gray)
        change = 0
      else:
        ## if there are face in image, narrow the detected are near to the face
        (x1, y1, x2, y2) = rects[0]
        xx1 = x1-20 if x1>20 else 0
        yy1 = y1-20 if y1>20 else 0
        xx2 = x2+20 if x2+20 < 640 else 640
        yy2 = y2+20 if y2+20 < 480 else 480
        # print('Get face at: ', rects[0], 'New detecting grid is:', [xx1, yy1, xx2, yy2])
        gray = cv.equalizeHist(gray[yy1:yy2, xx1:xx2])
        change = 1

      ## get time of start detectation
      t = clock()
      ## detect giving image by model
      rects = detect(gray, cascade)

      ## Drawing detected face on image
      if len(rects) != 0:
        ## Restore coordinates to original size
        if change:
          # print('Detected index from smaller grid:', x1, y1, x2, y2)
          x1, y1, x2, y2 = rects[0]
          x1, y1, x2, y2 = x1+xx1, y1+yy1, x2+xx1, y2+yy1
          rects = [[x1, y1, x2, y2]]
          # print('Re-index to oranginal image:', rects[0])
      
        draw_rects(img, rects, (0, 255, 0))
      # else:
        # print('No face!')
      dt = clock() - t

      draw_str(img, (20, 20), 'time: %.1f ms' % (dt*1000))
      print('time: %.1f ms' % (dt*1000))

      eval_js('showimg("{}")'.format(image2byte(img)))

      if cv.waitKey(5) == 27:
          break

    print('Done')

Excuting Face-Detection

In [ ]:
main()

NameError: ignored